## Wczytanie i przygotowanie danych

In [2]:
import pandas as pd
import numpy as np

In [3]:
# wczytanie zbioru w postaci linku do danych; testowo wybór dwóch systemów kodowania
link_do_danych = "https://raw.githubusercontent.com/saimadhu-polamuri/DataHakthon3X/master/dataSet/Train.csv"
system_kodowania1 = "ISO-8859-1"
system_kodowania2 = "latin"

df1_ = pd.read_csv(link_do_danych, header=0, index_col = 'ID', encoding = system_kodowania1,
                    converters={'Employer_Name': str, 'Salary_Account': str, 'City': str, 'DOB': str}, 
                    parse_dates=['Lead_Creation_Date'])
df2_ = pd.read_csv(link_do_danych, header=0, index_col = 'ID', encoding = system_kodowania2,
                    converters={'Employer_Name': str, 'Salary_Account': str, 'City': str, 'DOB': str}, 
                    parse_dates=['Lead_Creation_Date'])

# uwaga, nie parsuję automatycznie daty urodzenia 'DOB' gdyż read_csv wczytuje lata "dwucyforwe" do roku 68 jak 20XX,
# zamiast (w tym przypadku) jako 19XX

Dalsza analiza prowadzona jest dla danych w: 'df1_'

In [4]:
# usunięcie zmiennej (kolumny): LoggedIn zgodnie z wymogami projektu
df1_ = df1_.drop(columns=["LoggedIn"])

In [5]:
# modyfikacja parametrów wyświetlania liczby kolumn i wierszy z ramki pandas w celu przeglądu danych
with pd.option_context('display.max_rows', 20, 'display.max_columns', 999):
    display(df1_.head(5))

,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,Salary_Account,Mobile_Verified,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,Disbursed
ID,,,,,,,,,,,,,,,,,,,,,,,,
ID000002C20,Female,Delhi,20000,23-May-78,2015-05-15,300000.0,5.0,0.0,CYBOSOL,HDFC Bank,N,0,HBXX,NaN,NaN,NaN,NaN,NaN,N,Web-browser,G,S122,1,0
ID000004E40,Male,Mumbai,35000,07-Oct-85,2015-05-04,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000.0,2.0,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0
ID000007H20,Male,Panchkula,22500,10-Oct-81,2015-05-19,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000.0,4.0,NaN,NaN,NaN,N,Web-browser,B,S143,1,0
ID000008I30,Male,Saharsa,35000,30-Nov-87,2015-05-09,1000000.0,5.0,0.0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000.0,5.0,NaN,NaN,NaN,N,Web-browser,B,S143,3,0
ID000009J40,Male,Bengaluru,100000,17-Feb-84,2015-05-20,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000.0,2.0,NaN,NaN,NaN,N,Web-browser,B,S134,3,0


In [6]:
# przegląd informacji o danych w df1
df1_.info(20)

<class 'pandas.core.frame.DataFrame'>
Index: 87020 entries, ID000002C20 to ID124821V10
Data columns (total 24 columns):
Gender                   87020 non-null object
City                     87020 non-null object
Monthly_Income           87020 non-null int64
DOB                      87020 non-null object
Lead_Creation_Date       87020 non-null datetime64[ns]
Loan_Amount_Applied      86949 non-null float64
Loan_Tenure_Applied      86949 non-null float64
Existing_EMI             86949 non-null float64
Employer_Name            87020 non-null object
Salary_Account           87020 non-null object
Mobile_Verified          87020 non-null object
Var5                     87020 non-null int64
Var1                     87020 non-null object
Loan_Amount_Submitted    52407 non-null float64
Loan_Tenure_Submitted    52407 non-null float64
Interest_Rate            27726 non-null float64
Processing_Fee           27420 non-null float64
EMI_Loan_Submitted       27726 non-null float64
Filled_Form         

1. Poniżej przekształcam wszystkie daty wg reguły: ROK = 1900 + rr, gdzie rr to dwucyfrowy rok (typ int) wyodrębniony z pola DOB, zawierającego datę urodzenia w postaci string. W analizowanym zbiorze danych problem z datą dotyczy lat 50 - 68, które zamieniam na 1950-1968, podczas gdy read_csv wczytywałby je z automatu jako 2050 - 2068
2. Następnie z tak uzyskanej daty obliczam przybliżony wiek w latach dla każdego wnioskującego o kredyt poprzez odjęcie od daty: Lead_Creation_Date jego daty urodzenia: DOBd (już w formacie daty)
3. Dodatkowy warunek dotyczy sytuacji, w której różnica w latach = 100 (sytuacja ta ma mieksce dla kilkunstu rekordów dla których data Lead_Creation_Date oraz data urodzenia są sobie równe) Wtedy taktujmy to za błąd przy wpisywaniu danych i wstawiamy jako wiek: NaN 

In [7]:
from datetime import datetime
from dateutil import relativedelta

DOBdlst = [] # lista, do której wpiszę poprawnie wyliczone daty urodzenia już w formacie daty
Agelst = [] # lista, do której wpiszę wiek wnioskodawcy wyrażony w latach (od Lead_Creation_Date odejmiemy datę urodzenia)

for i in range (0, len(df1_)):
    DOBstr = df1_['DOB'][i]
    rr = int(DOBstr[-2:])
    rok = 1900 + rr # wyliczenie roku wg podanych wyżej założeń
    DOBstr = DOBstr[0:7]+str(rok) # złączenie roku 4-cyfrowego z dniem i miesiącem daty urodzenia
    DOBdate = datetime.strptime(DOBstr, '%d-%b-%Y') # przekształcenie daty urodzenia na format daty
    DOBdlst.append(DOBdate) # dołączenie danej daty do listy z datami urodzenia
    date_L = df1_['Lead_Creation_Date'][i]
    difference = relativedelta.relativedelta(date_L, DOBdate) # odjęcie od Lead_Creation_Date daty urodzenia wnioskodawcy
    years = difference.years
    if years != 100: # jest kilkanaście rekordów gdzie daty urodzenia i składania wniosku
                     # są sobie równe, co jest niemożliwe, 
                     # ktoś pomylił sie przy wpisywaniu danych
        Agelst.append(years)
    else:
        #print(years)
        Agelst.append(np.nan) # gdy różnica w datach = 100 wstawiamy wartość NaN

# utworzenie nowych pól w df1_ w oparciu o w/w listy
df1_['DOBd'] = DOBdlst
df1_['estimated_age_in_years'] = Agelst

In [8]:
# sprawdzenie efektów zastosowania powyższego algorytmu
kontrolnie1 = df1_.estimated_age_in_years.value_counts().sort_values()
kontrolnie1

71.0       1
86.0       1
78.0       1
66.0       1
82.0       1
77.0       1
69.0       3
73.0       4
72.0       4
68.0       5
67.0       6
65.0      12
62.0      22
17.0      25
61.0      27
63.0      28
64.0      42
60.0      49
59.0      81
58.0     102
56.0     130
57.0     148
18.0     189
55.0     234
54.0     240
53.0     281
52.0     297
19.0     324
51.0     325
50.0     360
49.0     367
48.0     385
46.0     474
47.0     476
44.0     556
43.0     616
20.0     705
42.0     769
41.0     803
45.0     987
40.0    1047
39.0    1179
38.0    1383
21.0    1402
37.0    1617
36.0    1929
22.0    2658
35.0    2664
34.0    3270
33.0    3603
23.0    4070
32.0    4170
31.0    4779
24.0    5290
30.0    5588
29.0    6220
28.0    6485
25.0    6627
27.0    6959
26.0    6981
Name: estimated_age_in_years, dtype: int64

In [9]:
df1_.head()

,Gender,City,Monthly_Income,DOB,Lead_Creation_Date,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,Salary_Account,...,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,Disbursed,DOBd,estimated_age_in_years
ID,,,,,,,,,,,,,,,,,,,,,
ID000002C20,Female,Delhi,20000,23-May-78,2015-05-15,300000.0,5.0,0.0,CYBOSOL,HDFC Bank,...,NaN,NaN,N,Web-browser,G,S122,1,0,1978-05-23,36.0
ID000004E40,Male,Mumbai,35000,07-Oct-85,2015-05-04,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,...,NaN,6762.9,N,Web-browser,G,S122,3,0,1985-10-07,29.0
ID000007H20,Male,Panchkula,22500,10-Oct-81,2015-05-19,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,...,NaN,NaN,N,Web-browser,B,S143,1,0,1981-10-10,33.0
ID000008I30,Male,Saharsa,35000,30-Nov-87,2015-05-09,1000000.0,5.0,0.0,BIHAR GOVERNMENT,State Bank of India,...,NaN,NaN,N,Web-browser,B,S143,3,0,1987-11-30,27.0
ID000009J40,Male,Bengaluru,100000,17-Feb-84,2015-05-20,500000.0,2.0,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,...,NaN,NaN,N,Web-browser,B,S134,3,0,1984-02-17,31.0


In [10]:
# zapis danych lokalnie na dysk (wykorzystywany w początkowej fazie analizy)
#df1_.to_csv(path_or_buf = 'dane/df1_d1.csv', encoding='utf-8')

Dalsza analiza prowadzona jest dla danych w: _df1_

In [11]:
#usunięcie zbędnych kolumn z datami, zostawiam tylko wiek
df1 = df1_.drop(columns=["Lead_Creation_Date", "DOB", "DOBd"])
df1.head(3)

,Gender,City,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,Salary_Account,Mobile_Verified,Var5,...,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,Disbursed,estimated_age_in_years
ID,,,,,,,,,,,,,,,,,,,,,
ID000002C20,Female,Delhi,20000,300000.0,5.0,0.0,CYBOSOL,HDFC Bank,N,0,...,NaN,NaN,NaN,N,Web-browser,G,S122,1,0,36.0
ID000004E40,Male,Mumbai,35000,200000.0,2.0,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,...,13.25,NaN,6762.9,N,Web-browser,G,S122,3,0,29.0
ID000007H20,Male,Panchkula,22500,600000.0,4.0,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,...,NaN,NaN,NaN,N,Web-browser,B,S143,1,0,33.0


In [12]:
# wyświeltenie informacji o typach zmiennych w ramce: df1
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87020 entries, ID000002C20 to ID124821V10
Data columns (total 23 columns):
Gender                    87020 non-null object
City                      87020 non-null object
Monthly_Income            87020 non-null int64
Loan_Amount_Applied       86949 non-null float64
Loan_Tenure_Applied       86949 non-null float64
Existing_EMI              86949 non-null float64
Employer_Name             87020 non-null object
Salary_Account            87020 non-null object
Mobile_Verified           87020 non-null object
Var5                      87020 non-null int64
Var1                      87020 non-null object
Loan_Amount_Submitted     52407 non-null float64
Loan_Tenure_Submitted     52407 non-null float64
Interest_Rate             27726 non-null float64
Processing_Fee            27420 non-null float64
EMI_Loan_Submitted        27726 non-null float64
Filled_Form               87020 non-null object
Device_Type               87020 non-null object
Var2      

In [13]:
# przegląd wartości wybranych zmiennych
df1.Gender.value_counts()

Male      49848
Female    37172
Name: Gender, dtype: int64

In [14]:
df1.Filled_Form.value_counts()

N    67530
Y    19490
Name: Filled_Form, dtype: int64

In [15]:
df1.Var1.value_counts()

HBXX    59294
HBXC     9010
HBXB     4479
HAXA     2909
HBXA     2123
HAXB     2011
HBXD     1964
HAXC     1536
HBXH      970
HCXF      722
HAYT      508
HAVC      384
HAXM      268
HCXD      237
HCYS      217
HVYS      186
HAZD      109
HCXG       78
HAXF       15
Name: Var1, dtype: int64

In [16]:
df1.Var2.value_counts()

B    37280
G    33032
C    14210
E     1315
D      634
F      544
A        5
Name: Var2, dtype: int64

In [17]:
df1.Var4.value_counts()

3    25260
1    23906
5    20266
4     6577
2     5931
0     2546
7     2302
6      232
Name: Var4, dtype: int64

In [18]:
df1.Var5.value_counts()

0     29087
1     12236
3      6759
11     5204
2      4485
14     3662
15     3509
12     2989
13     2622
8      2515
10     2427
9      2281
16     2097
4      1815
17     1691
7      1489
6       983
5       975
18      194
Name: Var5, dtype: int64

In [19]:
df1.City.value_counts()

Delhi                  12527
Bengaluru              10824
Mumbai                 10795
Hyderabad               7272
Chennai                 6916
Pune                    5207
Kolkata                 2888
Ahmedabad               1788
Jaipur                  1331
Gurgaon                 1212
Coimbatore              1147
                        1003
Thane                    905
Chandigarh               870
Surat                    802
Visakhapatnam            764
Indore                   734
Vadodara                 624
Nagpur                   594
Lucknow                  580
Ghaziabad                560
Bhopal                   513
Kochi                    492
Patna                    461
Faridabad                447
Madurai                  375
Noida                    373
Gautam Buddha Nagar      338
Dehradun                 314
Raipur                   289
                       ...  
Chinnamiram                1
Dantewada                  1
GANDEVI                    1
KHAMBHAT      

In [20]:
df1.Employer_Name.value_counts()

0                                               4914
TATA CONSULTANCY SERVICES LTD (TCS)              550
COGNIZANT TECHNOLOGY SOLUTIONS INDIA PVT LTD     404
ACCENTURE SERVICES PVT LTD                       324
GOOGLE                                           301
HCL TECHNOLOGIES LTD                             250
ICICI BANK LTD                                   239
INDIAN AIR FORCE                                 191
INFOSYS TECHNOLOGIES                             181
GENPACT                                          179
IBM CORPORATION                                  173
INDIAN ARMY                                      171
TYPE SLOWLY FOR AUTO FILL                        162
WIPRO TECHNOLOGIES                               155
HDFC BANK LTD                                    148
IKYA HUMAN CAPITAL SOLUTIONS LTD                 142
STATE GOVERNMENT                                 134
INDIAN RAILWAY                                   130
INDIAN NAVY                                   

In [21]:
df1.Device_Type.value_counts()

Web-browser    64316
Mobile         22704
Name: Device_Type, dtype: int64

In [22]:
df1.Source.value_counts()

S122    38567
S133    29885
S159     5599
S143     4332
S127     1931
S137     1724
S134     1301
S161      769
S151      720
S157      650
S153      494
S156      308
S144      299
S158      208
S123       73
S141       57
S162       36
S124       24
S160       11
S150       10
S155        4
S129        3
S136        3
S139        3
S138        3
S135        2
S130        1
S140        1
S125        1
S154        1
Name: Source, dtype: int64

In [23]:
df1.Salary_Account.value_counts()

HDFC Bank                                          17695
ICICI Bank                                         13636
State Bank of India                                11843
                                                   11764
Axis Bank                                           8783
Citibank                                            2376
Kotak Bank                                          2067
IDBI Bank                                           1550
Punjab National Bank                                1201
Bank of India                                       1170
Bank of Baroda                                      1126
Standard Chartered Bank                              995
Canara Bank                                          990
Union Bank of India                                  951
Yes Bank                                             779
ING Vysya                                            678
Corporation bank                                     649
Indian Overseas Bank           

In [24]:
df1.Mobile_Verified.value_counts()

Y    56481
N    30539
Name: Mobile_Verified, dtype: int64

Na podstawie analizy nazw pól oraz dostępnej informacji w internecie, przyjmuję, że jesli dla danego wnioskodawcy, pola: 'Loan_Amount_Submitted', 'Loan_Tenure_Submitted', 'EMI_Loan_Submitted' są puste to do finalnej analizy wniosku kredytowego brano pod uwagę wartości z pól: 'Loan_Amount_Applied' oraz 'Loan_Tenure_Applied', 'Existing_EMI'
W związku z tym poniżej przenoszę odpowiednie wartości z pól "applied" do "submited" (jeśli są puste) i w dalszej części biorę pod uwagę już tylko pola "submited", a 'applied' usuwam.

In [25]:
df1['Loan_Amount_Submitted'].fillna(df1['Loan_Amount_Applied'], inplace=True)
df1['Loan_Tenure_Submitted'].fillna(df1['Loan_Tenure_Applied'], inplace=True)
df1['EMI_Loan_Submitted'].fillna(df1['Existing_EMI'], inplace=True)

In [26]:
# usuwam pola typu 'Applied' oraz 'Existing Emi'
df1 = df1.drop(columns=["Loan_Tenure_Applied", "Loan_Amount_Applied", "Existing_EMI"])

In [27]:
# usuwam wiersze z danymi NaN dla zmiennej: Loan_Amount_Submited
df1.dropna(axis=0, subset=["Loan_Amount_Submitted", "EMI_Loan_Submitted"], inplace=True)

In [28]:
# usuwam wiersze z danymi NaN dla zmiennej: Existing_EMI - używane tylko podczas testów
#df1.dropna(axis=0, subset=['Existing_EMI'], inplace=True)

In [29]:
#Sprawdzenie ile jest danych z brakującymi wartościami
df1.isnull().sum()

Gender                        0
City                          0
Monthly_Income                0
Employer_Name                 0
Salary_Account                0
Mobile_Verified               0
Var5                          0
Var1                          0
Loan_Amount_Submitted         0
Loan_Tenure_Submitted         0
Interest_Rate             59252
Processing_Fee            59558
EMI_Loan_Submitted            0
Filled_Form                   0
Device_Type                   0
Var2                          0
Source                        0
Var4                          0
Disbursed                     0
estimated_age_in_years       17
dtype: int64

In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86978 entries, ID000002C20 to ID124821V10
Data columns (total 20 columns):
Gender                    86978 non-null object
City                      86978 non-null object
Monthly_Income            86978 non-null int64
Employer_Name             86978 non-null object
Salary_Account            86978 non-null object
Mobile_Verified           86978 non-null object
Var5                      86978 non-null int64
Var1                      86978 non-null object
Loan_Amount_Submitted     86978 non-null float64
Loan_Tenure_Submitted     86978 non-null float64
Interest_Rate             27726 non-null float64
Processing_Fee            27420 non-null float64
EMI_Loan_Submitted        86978 non-null float64
Filled_Form               86978 non-null object
Device_Type               86978 non-null object
Var2                      86978 non-null object
Source                    86978 non-null object
Var4                      86978 non-null int64
Disbursed     

Dla: Gender, Mobile_Verified, Device_Type używam pd.get_dummies (poniżej)

In [31]:
# podział danych w kolumnach wg wartości
dummies = pd.get_dummies(df1[['Filled_Form', 'Gender', 'Mobile_Verified', 'Device_Type']])

In [32]:
dummies.head(2)

,Filled_Form_N,Filled_Form_Y,Gender_Female,Gender_Male,Mobile_Verified_N,Mobile_Verified_Y,Device_Type_Mobile,Device_Type_Web-browser
ID,,,,,,,,
ID000002C20,1,0,1,0,1,0,0,1
ID000004E40,1,0,0,1,0,1,0,1


In [33]:
# usunięcie części zbędnych kolumn z df1
tmp1 = df1.drop(columns=["Filled_Form", "Gender", "Mobile_Verified", "Device_Type"])

Dalsze przetwarzanie dla Xf1

In [34]:
# połączenie tmp1 z dummies, przy czym jako, że w poszczególnych kolumnach znajdowały się tylko dwie wartości
# wybieram jedną z nich: _Y oraz _Mobile
Xf1 = pd.concat([tmp1, dummies[['Gender_Male', 'Filled_Form_Y', 'Mobile_Verified_Y', 'Device_Type_Mobile']]], axis = 1)

In [35]:
Xf1.head()

,City,Monthly_Income,Employer_Name,Salary_Account,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Var2,Source,Var4,Disbursed,estimated_age_in_years,Gender_Male,Filled_Form_Y,Mobile_Verified_Y,Device_Type_Mobile
ID,,,,,,,,,,,,,,,,,,,,
ID000002C20,Delhi,20000,CYBOSOL,HDFC Bank,0,HBXX,300000.0,5.0,NaN,NaN,0.0,G,S122,1,0,36.0,0,0,0,0
ID000004E40,Mumbai,35000,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,13,HBXA,200000.0,2.0,13.25,NaN,6762.9,G,S122,3,0,29.0,1,0,1,0
ID000007H20,Panchkula,22500,ALCHEMIST HOSPITALS LTD,State Bank of India,0,HBXX,450000.0,4.0,NaN,NaN,0.0,B,S143,1,0,33.0,1,0,1,0
ID000008I30,Saharsa,35000,BIHAR GOVERNMENT,State Bank of India,10,HBXX,920000.0,5.0,NaN,NaN,0.0,B,S143,3,0,27.0,1,0,1,0
ID000009J40,Bengaluru,100000,GLOBAL EDGE SOFTWARE,HDFC Bank,17,HBXX,500000.0,2.0,NaN,NaN,25000.0,B,S134,3,0,31.0,1,0,1,0


Labelizacja zmiennych kategoryzujących, alfanumerycznych: City, Employer_Name, Salary_Account, Source przy użyciu:LabelEncoder

In [36]:
# zdefiniowanie listy ze zmiennych do labelizacji
Vars_To_Labelize = ["City", "Employer_Name", "Salary_Account", "Source", "Var1", "Var2"]

In [37]:
from sklearn import preprocessing

def labelize(df, VarsToLabelize):
    
    """
    Parametry:
        df (Pandas Dataframe): zbiór danych w postaci dataframe,
            zawierający m. in. kolumny z danymi do "etykietowania"
        VarsToLabelize (list): lista zmiennych, którym chcemy przypisać etykiety 
            numeryczne przy wykorzystaniu LabelEncoder()
    Returns (zwraca): 
        zbiór danych df, zawierający wygenerowane przez LabelEncoder()
        etykiety numeryczne dla podanych kolumn w zmiennej VarsToLabelize
    w ramce danych: df
    """
    # inicalizacja LabelEncoder'a
    lbencdr = preprocessing.LabelEncoder()
    labels_str = '_labels'
    for labelVar in VarsToLabelize:
        lbencdr.fit(df[labelVar])
        labelVar_label = labelVar + labels_str
        df[labelVar_label] = lbencdr.transform(df[labelVar])
    return
        

In [38]:
labelize(Xf1, Vars_To_Labelize)
Xf1.head()

,City,Monthly_Income,Employer_Name,Salary_Account,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,...,Gender_Male,Filled_Form_Y,Mobile_Verified_Y,Device_Type_Mobile,City_labels,Employer_Name_labels,Salary_Account_labels,Source_labels,Var1_labels,Var2_labels
ID,,,,,,,,,,,,,,,,,,,,,
ID000002C20,Delhi,20000,CYBOSOL,HDFC Bank,0,HBXX,300000.0,5.0,NaN,NaN,...,0,0,0,0,173,8692,21,0,13,6
ID000004E40,Mumbai,35000,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,13,HBXA,200000.0,2.0,13.25,NaN,...,1,0,1,0,447,38691,23,0,8,6
ID000007H20,Panchkula,22500,ALCHEMIST HOSPITALS LTD,State Bank of India,0,HBXX,450000.0,4.0,NaN,NaN,...,1,0,1,0,498,1834,45,16,13,1
ID000008I30,Saharsa,35000,BIHAR GOVERNMENT,State Bank of India,10,HBXX,920000.0,5.0,NaN,NaN,...,1,0,1,0,569,5699,45,16,13,1
ID000009J40,Bengaluru,100000,GLOBAL EDGE SOFTWARE,HDFC Bank,17,HBXX,500000.0,2.0,NaN,NaN,...,1,0,1,0,88,13514,21,8,13,1


In [39]:
#usunięcie zmiennych poddanych labelizacji
Xf1 = Xf1.drop(columns=Vars_To_Labelize)

In [40]:
Xf1.head()

,Monthly_Income,Var5,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Var4,Disbursed,estimated_age_in_years,Gender_Male,Filled_Form_Y,Mobile_Verified_Y,Device_Type_Mobile,City_labels,Employer_Name_labels,Salary_Account_labels,Source_labels,Var1_labels,Var2_labels
ID,,,,,,,,,,,,,,,,,,,,
ID000002C20,20000,0,300000.0,5.0,NaN,NaN,0.0,1,0,36.0,0,0,0,0,173,8692,21,0,13,6
ID000004E40,35000,13,200000.0,2.0,13.25,NaN,6762.9,3,0,29.0,1,0,1,0,447,38691,23,0,8,6
ID000007H20,22500,0,450000.0,4.0,NaN,NaN,0.0,1,0,33.0,1,0,1,0,498,1834,45,16,13,1
ID000008I30,35000,10,920000.0,5.0,NaN,NaN,0.0,3,0,27.0,1,0,1,0,569,5699,45,16,13,1
ID000009J40,100000,17,500000.0,2.0,NaN,NaN,25000.0,3,0,31.0,1,0,1,0,88,13514,21,8,13,1


In [41]:
Xf1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86978 entries, ID000002C20 to ID124821V10
Data columns (total 20 columns):
Monthly_Income            86978 non-null int64
Var5                      86978 non-null int64
Loan_Amount_Submitted     86978 non-null float64
Loan_Tenure_Submitted     86978 non-null float64
Interest_Rate             27726 non-null float64
Processing_Fee            27420 non-null float64
EMI_Loan_Submitted        86978 non-null float64
Var4                      86978 non-null int64
Disbursed                 86978 non-null int64
estimated_age_in_years    86961 non-null float64
Gender_Male               86978 non-null uint8
Filled_Form_Y             86978 non-null uint8
Mobile_Verified_Y         86978 non-null uint8
Device_Type_Mobile        86978 non-null uint8
City_labels               86978 non-null int64
Employer_Name_labels      86978 non-null int64
Salary_Account_labels     86978 non-null int64
Source_labels             86978 non-null int64
Var1_labels           

In [42]:
Xf1.shape

(86978, 20)

In [43]:
#Sprawdzenie ile jest danych z brakującymi wartościami
Xf1.isnull().sum()

Monthly_Income                0
Var5                          0
Loan_Amount_Submitted         0
Loan_Tenure_Submitted         0
Interest_Rate             59252
Processing_Fee            59558
EMI_Loan_Submitted            0
Var4                          0
Disbursed                     0
estimated_age_in_years       17
Gender_Male                   0
Filled_Form_Y                 0
Mobile_Verified_Y             0
Device_Type_Mobile            0
City_labels                   0
Employer_Name_labels          0
Salary_Account_labels         0
Source_labels                 0
Var1_labels                   0
Var2_labels                   0
dtype: int64

In [44]:
# dodanie wartości średnich do poniższych kolumn zawierających dane NaN
Xf1['Interest_Rate'].fillna(Xf1['Interest_Rate'].mean(), inplace=True)
Xf1['Processing_Fee'].fillna(Xf1['Processing_Fee'].mean(), inplace=True)
Xf1['estimated_age_in_years'].fillna(Xf1['estimated_age_in_years'].mean(), inplace=True)

In [45]:
# Sprawdzenie jaki jest rozkłąd wartości zmiennej Disbursed
Xf1.Disbursed.value_counts()

0    85705
1     1273
Name: Disbursed, dtype: int64

## Widać, że wartości: 0 jest wielokrotnie wiecej niż 1 
## Rozkład nie jest zbalansowany

In [46]:
#Zdefiniowanie zmiennych zawierających liczbę 0 i 1 w zmiennej Disbursed
Disbursed_liczba_wystapien_0, Disbursed_liczba_wystapien_1 = Xf1.Disbursed.value_counts()[0], Xf1.Disbursed.value_counts()[1]
Disbursed_liczba_wystapien_0, Disbursed_liczba_wystapien_1

(85705, 1273)

In [47]:
#wyliczenie w przyblizeniu mnoznika = ile razy będę duplikował dane ze zbioru z samymi wartościami "1" w zmiennej Disbursed
# oraz reszty => tak by liczba obserwacji z "1" w Disbursed * mnożnik + reszta równała się liczbie obserwacji z "0" w Disbursed)
mnoznik = int(Disbursed_liczba_wystapien_0/Disbursed_liczba_wystapien_1)
reszta = Disbursed_liczba_wystapien_0-mnoznik*Disbursed_liczba_wystapien_1
mnoznik, reszta

(67, 414)

In [48]:
# wylosowanie ze zbioru danych zawierajacych 1, liczby obserwacji tylko z wartością "1" dla zmiennej Disbursed
# losujemy maksymalnie dostępną liczbę obserwacji tyle razy ile wynosi 'mnożnik' (czyli 66 razy)
# zapisanie tych danych do dataframe: X_Disbursed_1

# tu pierwsze samplowanie
X_Disbursed_1 = Xf1.sample(n=Disbursed_liczba_wystapien_1, weights='Disbursed', random_state=42)
# tu dodanie samplowania dla liczby obserwacji równej reszta = 414
Xprim = Xf1.sample(n=reszta, weights='Disbursed', random_state=42)
X_Disbursed_1 = pd.concat([X_Disbursed_1, Xprim], ignore_index=True)

# poniżej generowanie pozostałej liczby wymaganych obserwacji w pętli
for i in range(0, mnoznik-2):
    Xbis = Xf1.sample(n=Disbursed_liczba_wystapien_1, weights='Disbursed', random_state=42)
    X_Disbursed_1 = pd.concat([X_Disbursed_1, Xbis], ignore_index=True)


In [49]:
# Sprawdzenie rozkładu wartości zmiennej Disbursed dla wylosowanych danych
X_Disbursed_1.Disbursed.value_counts()

1    84432
Name: Disbursed, dtype: int64

In [50]:
X_Disbursed_1.head()

,Monthly_Income,Var5,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Var4,Disbursed,estimated_age_in_years,Gender_Male,Filled_Form_Y,Mobile_Verified_Y,Device_Type_Mobile,City_labels,Employer_Name_labels,Salary_Account_labels,Source_labels,Var1_labels,Var2_labels
0,95000,12,300000.0,4.0,19.197474,5131.150839,9000.0,2,1,32.0,1,0,1,0,95,38965,21,0,13,1
1,132000,13,1500000.0,5.0,19.197474,5131.150839,10000.0,3,1,29.0,1,0,1,0,173,2158,21,0,13,6
2,36000,7,300000.0,4.0,19.197474,5131.150839,7206.0,3,1,27.0,1,0,1,0,695,37668,23,0,13,6
3,43000,10,300000.0,4.0,19.197474,5131.150839,0.0,3,1,28.0,1,0,1,0,463,23871,39,7,13,1
4,54000,15,300000.0,2.0,19.197474,5131.150839,0.0,3,1,37.0,1,0,1,0,524,32667,23,7,13,1


In [51]:
Xf1 = pd.concat([Xf1, X_Disbursed_1], ignore_index=True)

In [52]:
Xf1.Disbursed.value_counts()

1    85705
0    85705
Name: Disbursed, dtype: int64

## Zdefiniowanie finalnej postaci zmiennej X i y

In [53]:
# zdefiniowanie zmiennej y
y = Xf1.Disbursed

In [54]:
#usunięcie zmiennej y
X = Xf1.drop(columns="Disbursed")

## Załadowanie potrzebnych bibliotek ML

In [55]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer
import pprint
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix

## Podział na dane trenujące i testowe

In [56]:
# podział na dane trenujące i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
# Dodanie możliwości wyświetlania boldem w Pythonie-Jupyterze
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
printmd("**bold**")

**bold**

Sprawdzenie kształtów zbiorów train i test

In [58]:
X_train.shape

(137128, 19)

In [59]:
y_train.shape

(137128,)

In [60]:
X_test.shape

(34282, 19)

In [61]:
y_train.shape

(137128,)

In [62]:
y_test.shape

(34282,)

In [63]:
# spojrzenie na rozkład 0 i 1 dla y_train
y_train.value_counts()

0    68664
1    68464
Name: Disbursed, dtype: int64

In [64]:
# spojrzenie na rozkład 0 i 1 dla y_test
y_test.value_counts()

1    17241
0    17041
Name: Disbursed, dtype: int64

In [65]:
# ograniczenie informacji typu warning podczas testowania modeli
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [88]:
#definicje zmiennych, modeli oraz zakresu parametrów modeli, które będziemy tunować

names = np.array(["Naiwny Bayes", "Drzewo decyzyjne", "Regresja logistyczna_MaxAbs",
                  "Regresja logistyczna_StandardScaler",  "SVM_MaxAbs", "SVM_StandardScaler",
                 "BaggingClassifier_tree", "BaggingClassifier_Logistic", "RandomForest",
                  "bzprmRandomForest"])

models = [[("model", MultinomialNB())],
         [("model", DecisionTreeClassifier())],
         [("scaler", MaxAbsScaler()), ("model", LogisticRegression())],
          [("scaler", StandardScaler()), ("model", LogisticRegression())],
         [("scaler", MaxAbsScaler()),("model", SVC())],
          [("scaler", StandardScaler()),("model", SVC())],
         [("model", BaggingClassifier(base_estimator=DecisionTreeClassifier()))],
         [("model", BaggingClassifier(base_estimator=LogisticRegression()))],
         [("model", RandomForestClassifier())],
          [("model", RandomForestClassifier(n_estimators=180, min_samples_leaf=3,
                                            max_features=0.5, n_jobs=-1))]
         ]

param_grids = [{"model__alpha": [0.1, 1, 10], "model__fit_prior": [False, True]},
               {"model__criterion": ["gini", "entropy"],
                "model__min_samples_split": [2, 10, 100], "model__max_depth": [None, 2, 10, 100]}, 
               {"model__penalty": ["l1", "l2"], "model__C": [0.001, 0.1, 1, 10, 100]},
               {"model__penalty": ["l1", "l2"], "model__C": [0.001, 0.1, 1, 10, 100]},
              [{"model__kernel": ["rbf"],},
              {"model__kernel": ["poly"], "model__degree": [1, 2, 3, 4]},
              {"model__kernel": ["linear","sigmoid"]}],
               [{"model__kernel": ["rbf"],},
              {"model__kernel": ["poly"], "model__degree": [1, 2, 3, 4]},
              {"model__kernel": ["linear","sigmoid"]}],
               {"model__n_estimators" : [2, 5, 100], "model__max_features": [0.5, 0.8, 1.0]},
               {"model__n_estimators" : [2, 5, 100], "model__max_features": [0.5, 0.8, 1.0]},
               {"model__n_estimators" : [2, 5, 100]},
               {}
              ]


uses = np.array([True, True, True, True, False, False, True, True, True, True])
#uses = np.array([False, False, False, False, False, False, True, False, False, False])
#uses = np.array([True, True, True, True, True, True, True, True, True, True])
#uses = np.array([True, False, False, False, False, False, False, False, False, False])

In [89]:
# Wybór najlepszych modeli, prezentacja wyników


if len(names) != len(models) or len(models) != len(param_grids) or len(param_grids) != len(uses):
    print(f"len(names): {len(names)}")
    print(f"len(models): {len(models)}")
    print(f"len(param_grids): {len(param_grids)}")
    print(f"len(uses): {len(uses)}")
    raise ValueError("Listy nie mają tej samej długości!")

# puste listy do których wpisane zostana wybran eparametry i wyniki modeli
best_models = []
best_params = []
best_indexes = []
tab_ROC_AUC = []
tab_accuracy = []
names_rslt = [] # lista z nazwami modeli, które zostały przetestowane


for use, name, pipe, params in zip(uses, names, models, param_grids):
    if not use:
        continue
    printmd(f"**Tunuje model: {name}**") 
    pipeline = Pipeline(pipe)
    gs = GridSearchCV(estimator=pipeline, param_grid=params, n_jobs=3)
    gs.fit(X_train, y_train)
    for mean, std, param, fit_time, score_time in zip(gs.cv_results_["mean_test_score"],
                                gs.cv_results_["std_test_score"],
                                gs.cv_results_["params"],
                                gs.cv_results_["mean_fit_time"],
                                gs.cv_results_["mean_score_time"]):
        print("Zastosowane parametry:")
        for i in param.items():
            print(f"\t {i}")
        #print(f"\t {param}:\n\t mean_test_score: {mean}, std_test_score: {std},\n\t mean_fit_time: {fit_time}, mean_score_time: {score_time}\n")
        print(f"Wyniki: \n\t mean_test_score: {round(mean, 4)}, \
              std_test_score: {round(std, 4)}, \
              \n\t mean_fit_time: {round(fit_time, 4)}, \
              mean_score_time: {round(score_time, 4)}\n")
    names_rslt.append(name)
    best_models.append(gs.best_estimator_)
    best_params.append(gs.best_params_)
    best_indexes.append(gs.best_index_)
    
best_models = np.array(best_models)
best_params = np.array(best_params)
best_indexes = np.array(best_indexes)
names_rslt = np.array(names_rslt)


print(f"____________________\n\n")

#print(f"\n")
#printmd(f"**Testuję accuracy:**")
#for name, best_model in zip(names, best_models):
#    if not use:
#        continue
#    tab_accuracy.append(roc_auc_score(best_model.predict(X_test), y_test))
#    print(f"\t {name}: {round(accuracy_score(best_model.predict(X_test), y_test), 4)}")


**Tunuje model: Naiwny Bayes**

Zastosowane parametry:
	 ('model__alpha', 0.1)
	 ('model__fit_prior', False)
Wyniki: 
	 mean_test_score: 0.5388,               std_test_score: 0.0088,               
	 mean_fit_time: 0.281,               mean_score_time: 0.0645

Zastosowane parametry:
	 ('model__alpha', 0.1)
	 ('model__fit_prior', True)
Wyniki: 
	 mean_test_score: 0.5388,               std_test_score: 0.0088,               
	 mean_fit_time: 0.1843,               mean_score_time: 0.0383

Zastosowane parametry:
	 ('model__alpha', 1)
	 ('model__fit_prior', False)
Wyniki: 
	 mean_test_score: 0.5388,               std_test_score: 0.0088,               
	 mean_fit_time: 0.2033,               mean_score_time: 0.0466

Zastosowane parametry:
	 ('model__alpha', 1)
	 ('model__fit_prior', True)
Wyniki: 
	 mean_test_score: 0.5388,               std_test_score: 0.0088,               
	 mean_fit_time: 0.2058,               mean_score_time: 0.0316

Zastosowane parametry:
	 ('model__alpha', 10)
	 ('model__fit_prior', False)
Wyniki: 
	 

**Tunuje model: Drzewo decyzyjne**

Zastosowane parametry:
	 ('model__criterion', 'gini')
	 ('model__max_depth', None)
	 ('model__min_samples_split', 2)
Wyniki: 
	 mean_test_score: 0.9878,               std_test_score: 0.0004,               
	 mean_fit_time: 3.6246,               mean_score_time: 0.0747

Zastosowane parametry:
	 ('model__criterion', 'gini')
	 ('model__max_depth', None)
	 ('model__min_samples_split', 10)
Wyniki: 
	 mean_test_score: 0.9877,               std_test_score: 0.0001,               
	 mean_fit_time: 4.8119,               mean_score_time: 0.0563

Zastosowane parametry:
	 ('model__criterion', 'gini')
	 ('model__max_depth', None)
	 ('model__min_samples_split', 100)
Wyniki: 
	 mean_test_score: 0.9576,               std_test_score: 0.0015,               
	 mean_fit_time: 3.1466,               mean_score_time: 0.0443

Zastosowane parametry:
	 ('model__criterion', 'gini')
	 ('model__max_depth', 2)
	 ('model__min_samples_split', 2)
Wyniki: 
	 mean_test_score: 0.7272,               std_test_score: 0.0013,

**Tunuje model: Regresja logistyczna_MaxAbs**

...

Zastosowane parametry:
	 ('model__max_features', 0.5)
	 ('model__n_estimators', 2)
Wyniki: 
	 mean_test_score: 0.6847,               std_test_score: 0.0418,               
	 mean_fit_time: 4.84,               mean_score_time: 0.0509

Zastosowane parametry:
	 ('model__max_features', 0.5)
	 ('model__n_estimators', 5)
Wyniki: 
	 mean_test_score: 0.7254,               std_test_score: 0.0042,               
	 mean_fit_time: 8.9226,               mean_score_time: 0.0916

Zastosowane parametry:
	 ('model__max_features', 0.5)
	 ('model__n_estimators', 100)
Wyniki: 
	 mean_test_score: 0.7163,               std_test_score: 0.0027,               
	 mean_fit_time: 246.5619,               mean_score_time: 0.7097

Zastosowane parametry:
	 ('model__max_features', 0.8)
	 ('model__n_estimators', 2)
Wyniki: 
	 mean_test_score: 0.7225,               std_test_score: 0.0047,               
	 mean_fit_time: 13.1351,               mean_score_time: 0.0466

Zastosowane parametry:
	 ('model__max_features', 0.8)

**Tunuje model: RandomForest**

Zastosowane parametry:
	 ('model__n_estimators', 2)
Wyniki: 
	 mean_test_score: 0.9973,               std_test_score: 0.0003,               
	 mean_fit_time: 1.3641,               mean_score_time: 0.1264

Zastosowane parametry:
	 ('model__n_estimators', 5)
Wyniki: 
	 mean_test_score: 0.9962,               std_test_score: 0.0006,               
	 mean_fit_time: 2.9025,               mean_score_time: 0.154

Zastosowane parametry:
	 ('model__n_estimators', 100)
Wyniki: 
	 mean_test_score: 0.9992,               std_test_score: 0.0001,               
	 mean_fit_time: 48.9727,               mean_score_time: 2.9864



**Tunuje model: bzprmRandomForest**

Zastosowane parametry:
Wyniki: 
	 mean_test_score: 0.9975,               std_test_score: 0.0002,               
	 mean_fit_time: 183.3999,               mean_score_time: 6.0372

____________________




## Wyświetlenie wyników dla testowanych modeli

In [110]:
print(f"\n")
printmd(f"**Testuję ROC_AUC:**")
for name, best_model in zip(names_rslt, best_models):
    tab_ROC_AUC.append(roc_auc_score(best_model.predict(X_test), y_test))
    print(f"\t {name}: {round(roc_auc_score(best_model.predict(X_test), y_test), 5)}")
    
printmd(f"**Testuję accuracy:**")
for name, best_model in zip(names_rslt, best_models):
    tab_accuracy.append(accuracy_score(best_model.predict(X_test), y_test))
    printmd(f"\t\t {name}: {round(accuracy_score(best_model.predict(X_test), y_test), 5)}")
    # kontrolnie, przegląd confusion matrix, 
    # żeby sprawdzić jak modele radzą sobie z poprawną predycją dla Disbursed = "1"
    print(f"\t" "Kontrolnie: confusion matrix")
    cm = confusion_matrix(best_model.predict(X_test), y_test)
    print(f"\t {cm[0]}")
    print(f"\t {cm[1]}")
    #print(f"\n")

**Testuję ROC_AUC:**

	 Naiwny Bayes: 0.53445
	 Drzewo decyzyjne: 0.99218
	 Regresja logistyczna_MaxAbs: 0.72591
	 Regresja logistyczna_StandardScaler: 0.72594
	 BaggingClassifier_tree: 0.99927
	 BaggingClassifier_Logistic: 0.70993
	 RandomForest: 0.99965
	 bzprmRandomForest: 0.99904


**Testuję accuracy:**

		 Naiwny Bayes: 0.53425

	Kontrolnie: confusion matrix
	 [7988 6914]
	 [ 9053 10327]


...

	Kontrolnie: confusion matrix
	 [12071  4974]
	 [ 4970 12267]


		 RandomForest: 0.99965

	Kontrolnie: confusion matrix
	 [17029     0]
	 [   12 17241]


		 bzprmRandomForest: 0.99904

	Kontrolnie: confusion matrix
	 [17008     0]
	 [   33 17241]


## Umieszczenie wybranych parametrów modelu wraz z wynikami w ramce danych. 
## Posortowanie wyników malejąco względem ROC_AUC

In [120]:
#wpisanie wybranych parametrów i wyników testowanych modeli do ramki danych pandas
miary_wynikowe = ['BEST_PARAMS', 'ROC_AUC', 'ACCURACY']
zestawienie_wynikow = pd.DataFrame(list(zip(best_params, tab_ROC_AUC, tab_accuracy)), index = names_rslt, columns= miary_wynikowe)

pd.set_option('display.max_colwidth', 800)
zestawienie_wynikow.sort_values("ROC_AUC", ascending=False)

,BEST_PARAMS,ROC_AUC,ACCURACY
RandomForest,{'model__n_estimators': 100},0.999652,0.999650
BaggingClassifier_tree,"{'model__max_features': 0.5, 'model__n_estimators': 100}",0.999270,0.999271
bzprmRandomForest,{},0.999045,0.999037
Drzewo decyzyjne,"{'model__criterion': 'gini', 'model__max_depth': 100, 'model__min_samples_split': 2}",0.992178,0.992007
Regresja logistyczna_StandardScaler,"{'model__C': 0.1, 'model__penalty': 'l2'}",0.725937,0.725512
Regresja logistyczna_MaxAbs,"{'model__C': 100, 'model__penalty': 'l1'}",0.725909,0.725483
BaggingClassifier_Logistic,"{'model__max_features': 0.5, 'model__n_estimators': 5}",0.709925,0.709935
Naiwny Bayes,"{'model__alpha': 0.1, 'model__fit_prior': False}",0.534452,0.534245


## Poniżej są już tylko wyświetlenia kontrolne wybranych zmiennych

In [112]:
best_model.predict(X_test)

array([1, 0, 1, ..., 1, 1, 0])

In [113]:
#cm = confusion_matrix(best_model.predict(X_test), y_test)
#print(cm)

In [114]:
best_indexes

array([0, 9, 8, 3, 2, 1, 2, 0])

In [115]:
best_models

array([Pipeline(memory=None,
     steps=[('model', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=False))]),
       Pipeline(memory=None,
     steps=[('model', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       Pipeline(memory=None,
     steps=[('scaler', MaxAbsScaler(copy=True)), ('model', LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', Logi

In [116]:
best_params

array([{'model__alpha': 0.1, 'model__fit_prior': False},
       {'model__criterion': 'gini', 'model__max_depth': 100, 'model__min_samples_split': 2},
       {'model__C': 100, 'model__penalty': 'l1'},
       {'model__C': 0.1, 'model__penalty': 'l2'},
       {'model__max_features': 0.5, 'model__n_estimators': 100},
       {'model__max_features': 0.5, 'model__n_estimators': 5},
       {'model__n_estimators': 100}, {}], dtype=object)

In [117]:
tab_ROC_AUC

[0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309,
 0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309,
 0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309,
 0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309,
 0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309,
 0.5344521842671051,
 0.9921781330288324,
 0.725908539279977,
 0.7259366239841872,
 0.9992696249874815,
 0.7099254908018564,
 0.9996522343940184,
 0.9990448072247309

In [118]:
tab_accuracy

[0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957178694,
 0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957178694,
 0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957178694,
 0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957178694,
 0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957178694,
 0.5342453765824631,
 0.9920074674756432,
 0.7254827606324018,
 0.7255119304591331,
 0.9992707543317193,
 0.7099352429846567,
 0.9996499620792253,
 0.9990373957

In [119]:
#with pd.option_context('display.max_rows', 999, 'display.max_columns', 999):
#    display(X.head(10))